## Create Example - Rasterize Single Year of Fire Data

In [1]:
#https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python
# use env:gdal_env_2.7v.2
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np

Set up your filenames

In [2]:
shp_fn = r'../Data/Examples/fire17_1.geojson'

Open the file with GeoPANDAS read_file

In [3]:
fires = gpd.read_file(shp_fn)

CHeck and fix datatype issues

In [4]:
#print fires.columns
#fires['YEAR_'].head()
#fires.YEAR_.unique()

In [5]:
print fires.dtypes.head()

YEAR_        object
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


In [6]:
# subset to year and convert to integer
fires = fires[fires['YEAR_'].isin(['1980'])]
fires['YEAR_'] = fires['YEAR_'].astype(str).astype(int)
print fires.dtypes.head()

# create column of ones to rasterize for presence (1) of fire
fires['ONES'] = 1

YEAR_         int32
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


Open the raster file you want to use as a template for feature burning using rasterio


In [7]:
example = r'../Data/Examples/aet-198403.tif'

rst = rasterio.open(example)
rst.bounds

BoundingBox(left=-374495.83635354, bottom=-616363.3341887, right=566504.16364646, top=592636.6658113)

copy and update the metadata from the input raster for the output


In [8]:
with rasterio.open(example) as src:
    array = src.read()
    profile = src.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw',nodata=0)
    out_arr = src.read(1) # get data from first band, this gets updated in write
    print profile

{'count': 1, 'crs': CRS({u'lon_0': -120, u'ellps': u'GRS80', u'y_0': -4000000, u'no_defs': True, u'proj': u'aea', u'x_0': 0, u'units': u'm', u'towgs84': u'0,0,0,0,0,0,0', u'lat_2': 40.5, u'lat_1': 34, u'lat_0': 0}), 'interleave': 'band', 'dtype': 'float32', 'driver': u'GTiff', 'transform': Affine(1000.0, 0.0, -374495.83635354,
       0.0, -1000.0, 592636.6658113), 'height': 1209, 'width': 941, 'tiled': False, 'nodata': 0, 'compress': 'lzw'}


Now burn the features into the raster and write it out


In [9]:
output = r'../Data/Examples/fire_1980.tif' # any new file
    
# Write to tif, using the same profile as the source
with rasterio.open(output, 'w', **profile) as dst:
        
    # Write the product as a raster band to a new  file. For
    # the new file's profile, we start with the meta attributes of
    # the source file, but then change the band count to 1, set the
    # dtype to float, and specify LZW compression, missing = 0.

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(fires.geometry, fires.ONES))

    #rasterize shapes to raster values based on centroid 
    burned_value = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=dst.transform)
    # write out values 
    dst.write(burned_value,1)

The overall idea is to create an iterable containing tuples of (geometry, value), where the geometry is a shapely geometry and the value is what you want to burn into the raster at that geometry's location. Both Fiona and GeoPANDAS use shapely geometries so you are in luck there. In this example a generator is used to iterate through the (geometry,value) pairs which were extracted from the GeoDataFrame and joined together using zip().

## Create Annual Fire Rasters


In [27]:
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np


poly = r'../Data/Examples/fire17_1.geojson'
raster_exmpl = r'../Data/Examples/aet-198403.tif'
raster_out = r'../Data/Examples/fire_1980v2.tif' 
year_col_name='YEAR_'
year_sub_list=range(1980,1990) 

def fire_rasterizer(poly,raster_exmpl,raster_out,year_col_name='YEAR_',year_sub_list=range(1980,1990)):
    
    # subset to year and convert to integer
    fires = gpd.read_file(poly)
    fires = fires[fires[year_col_name].isin( [str(i) for i in year_sub_list] )]

    # create column of ones to rasterize for presence (1) of fire
    fires['ONES'] = 1
    fires.head(3)

    # get example metadata
    with rasterio.open(raster_exmpl) as src:
        array = src.read()
        profile = src.profile
        profile.update(dtype=rasterio.float32, count=1, compress='lzw',nodata=0)
        out_arr = src.read(1) # get data from first band, this gets updated in write

    # Write to tif, using the same profile as the source
    with rasterio.open(raster_out, 'w', **profile) as dst:

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(fires.geometry, fires.ONES))

        #rasterize shapes 
        burned_value = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=dst.transform)
        dst.write(burned_value,1)

fire_rasterizer(poly,raster_exmpl,raster_out,year_col_name='YEAR_',year_sub_list=range(1980,1990))
   